In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import scipy as sp
import matplotlib.pyplot as plt
import re
from os import listdir
from os.path import isfile, join
from scipy import stats

from sklearn import preprocessing

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectFromModel

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier

from sklearn.ensemble import VotingClassifier

# Evaluation Procedures
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score 
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import precision_recall_curve 
from sklearn.metrics import roc_auc_score
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import roc_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc

In [2]:
rare_AD = pd.read_csv('../Data/data_AD_rare_1319.csv')

In [3]:
pheno = pd.read_csv('../Data/phenotypes_1319.csv',sep=';')

In [4]:
rare_AD_1 = rare_AD.reset_index()
rare_AD_1.drop(columns=['index','Unnamed: 0'], inplace=True)
value_counts = rare_AD_1.apply(pd.Series.value_counts)
value_counts = value_counts.transpose()
value_counts.columns = value_counts.columns.astype(str)
value_counts = value_counts.reset_index()
value_counts.rename(columns={'index':'mutation'}, inplace=True)
value_counts['percentage of 1'] = value_counts.apply(lambda row : row['1']/(len(rare_AD_1)), axis=1)

In [5]:
rare_AD.drop(columns=['ARSD', 'ATP6AP1', 'FRMPD4', 'SHROOM4', 'SOX21', 'SLC24A5','NPIPB15', 'FAM9A', 'SUPT20HL1', 'VCX3A', 'VCX3B'], inplace=True, axis=1)
rare_AD.drop(columns=value_counts[value_counts['percentage of 1'] >= 0.0983].mutation, inplace=True, axis=1)
rare_AD.drop(columns=value_counts[value_counts['0'] >= len(rare_AD_1)].mutation, inplace=True, axis=1)
rare_AD.drop(columns=value_counts[value_counts['1'] >= len(rare_AD_1)].mutation, inplace=True, axis=1)

In [6]:
value_counts[value_counts['percentage of 1'] >= 0.0983]

,mutation,0,1,percentage of 1
1,AADACL3,660.0,658.0,0.499241
3,ABCA4,1122.0,196.0,0.148710
40,AGRN,1080.0,238.0,0.180577
73,ANKRD35,29.0,1289.0,0.977997
103,ASPM,1147.0,171.0,0.129742
...,...,...,...,...
17928,SPANXN3,81.0,1237.0,0.938543
17934,SRPX,1024.0,294.0,0.223065
18007,UBE2NL,800.0,518.0,0.393020
18020,VCX,1065.0,253.0,0.191958


Mantengo solo pazienti con un grading

In [7]:
pheno = pheno[pheno['grading_1319_adj_a_s'].notna()]

Scarto pazienti senza età

In [8]:
pheno = pheno[pheno['Age'].notna()]

Cambio l'indice dei pazienti

In [9]:
pheno.rename(columns={"UsedSampleCode":"key"}, inplace=True)
pheno.set_index('key',inplace=True)

In [10]:
len(pheno)

1312

Scarto pazienti da escludere

In [11]:
rare_AD.rename(columns={"Unnamed: 0": "key"}, inplace=True)
rare_AD.set_index('key',inplace=True)

rare_AD = rare_AD[rare_AD.index != 'COV2925-1241_hg38']
rare_AD = rare_AD[rare_AD.index != 'COV2928-1242_hg38']
rare_AD = rare_AD[rare_AD.index != 'COV3204-1334_hg38']
rare_AD = rare_AD[rare_AD.index != 'COV3196-1326_hg38']
rare_AD = rare_AD[rare_AD.index != 'COV2735-1159_hg38']
rare_AD = rare_AD[rare_AD.index != 'COV3211-1341_hg38']
rare_AD = rare_AD[rare_AD.index != 'COV2233-939_hg38']

rare_AD.index = rare_AD.index.str.replace('_hg38','')

In [12]:
grading = pheno[['Gender (M=0, F=1)','grading_1319_adj_a_s']]

df = grading.join(rare_AD, on='key')
df.rename(columns={'grading_1319_adj_a_s':'grading'}, inplace=True)

df = df[df['grading']!='none']
df.dropna(inplace=True)
df

,"Gender (M=0, F=1)",grading,A3GALT2,AADACL4,ABCB10,ABCD3,ABL2,ACADM,ACAP3,ACBD3,...,KDM5D,NLGN4Y,PCDH11Y,PRORY,RPS4Y2,TBL1Y,TMSB4Y,USP9Y,UTY,ZFY
key,,,,,,,,,,,,,,,,,,,,,
AR-COV-1,1.0,1,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AR-COV-10,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AR-COV-11,1.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AR-COV-13,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AR-COV-14,0.0,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
USCASI0004,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
USCASI0005,0.0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
VE10,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Mantengo solo i maschi

In [13]:
df['Gender (M=0, F=1)'].value_counts()

0.0    491
1.0    327
Name: Gender (M=0, F=1), dtype: int64

In [14]:
df = df[df['Gender (M=0, F=1)']==0].iloc[:,1:]

In [15]:
df = df.apply(pd.to_numeric, errors='coerce')
df = df.dropna()
df.reset_index(inplace=True)

In [16]:
df.drop(columns='key',inplace=True)

In [17]:
df

,grading,A3GALT2,AADACL4,ABCB10,ABCD3,ABL2,ACADM,ACAP3,ACBD3,ACBD6,...,KDM5D,NLGN4Y,PCDH11Y,PRORY,RPS4Y2,TBL1Y,TMSB4Y,USP9Y,UTY,ZFY
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
487,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
488,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
489,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Optimizing Lasso Classification 

In [18]:
target_variable = 'grading'
input_variables = df.columns[df.columns!=target_variable]

X = df[input_variables]
y = df[target_variable]
#X_nic = pd.read_csv('../Data/Male/X_antonio.csv')
#y_nic = pd.read_csv('../Data/Male/y_antonio.csv')['0']

In [19]:
#X_nic.drop(columns='Unnamed: 0', inplace= True)

In [20]:
#y_nic.drop(columns='Unnamed: 0', inplace=True)

In [21]:
#np.setdiff1d(X.columns, X_nic.columns).shape[0]

In [22]:
X.columns += '_rare_AD'

In [23]:
print("Class %2d  %.1f%%\nClass %2d  %.1f%%\n"%((y.value_counts()/y.shape[0]).index[0],100*(y.value_counts()/y.shape[0]).values[0],(y.value_counts()/y.shape[0]).index[1],100*(y.value_counts()/y.shape[0]).values[1]))

Class  0  51.7%
Class  1  48.3%



In [24]:
np.random.seed(1234)

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, train_size= 0.9, random_state=1234, shuffle=True)

crossvalidation = StratifiedKFold(n_splits=10, shuffle=True, random_state=1234)

In [25]:
c = np.logspace(np.log10(1e-2), np.log10(1e3), 51)

In [26]:
lasso = LogisticRegression(random_state=1234, solver="liblinear", penalty='l1',verbose=1)
parameters = {'C':c}
lasso_gs = GridSearchCV(lasso, parameters, n_jobs=-1, cv=crossvalidation, scoring='roc_auc')
lasso_gs.fit(X,y)

[LibLinear]

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=1234, shuffle=True),
             estimator=LogisticRegression(penalty='l1', random_state=1234,
                                          solver='liblinear', verbose=1),
             n_jobs=-1,
             param_grid={'C': array([1.00000000e-02, 1.25892541e-02, 1.58489319e-02, 1.99526231e-02,
       2.51188643e-02, 3.16227766e-02, 3.98107171e-02, 5.01187234e-02,
       6.30957344e-02, 7.94328235e-02, 1...
       6.30957344e+00, 7.94328235e+00, 1.00000000e+01, 1.25892541e+01,
       1.58489319e+01, 1.99526231e+01, 2.51188643e+01, 3.16227766e+01,
       3.98107171e+01, 5.01187234e+01, 6.30957344e+01, 7.94328235e+01,
       1.00000000e+02, 1.25892541e+02, 1.58489319e+02, 1.99526231e+02,
       2.51188643e+02, 3.16227766e+02, 3.98107171e+02, 5.01187234e+02,
       6.30957344e+02, 7.94328235e+02, 1.00000000e+03])},
             scoring='roc_auc')

In [27]:
s = {'score':lasso_gs.cv_results_['mean_test_score'],'std':lasso_gs.cv_results_['std_test_score'],'c':1/c}
scores = pd.DataFrame(data=s)

In [28]:
sns.set_theme()
plt.figure(figsize=(15,7))
ax = sns.lineplot(data=scores, x='c', y='score', ci=scores.std, markers=True, marker='o')
ax.fill_between(1/parameters['C'], y1=lasso_gs.cv_results_['mean_test_score'] - lasso_gs.cv_results_['std_test_score'], 
                y2=lasso_gs.cv_results_['mean_test_score'] + lasso_gs.cv_results_['std_test_score'], alpha=.2)
ax.set_xscale('log')
ax.set_ylabel('ROC AUC')
ax.set_xlabel('REGULARIZATION FACTOR')
ax.set_title('Lasso LogisticRegression GridSearch')
plt.plot(1/parameters['C'][11],lasso_gs.cv_results_['mean_test_score'][11], 'ok', markersize=5, color='red')
#ax.set_ylim([0.475,0.7])
plt.savefig('./Plots/GS_AD_rare_male.png', dpi=150)

AttributeError: module 'seaborn' has no attribute 'set_theme'

In [ ]:
lasso_gs.best_estimator_

In [ ]:
c_nicola = 0.125892541

In [ ]:
c[11]

In [29]:
method_name = 'Lasso'

#lasso = lasso_gs.best_estimator_
lasso = LogisticRegression(C=c[11], penalty='l1', random_state=1234, solver='liblinear', verbose=1)

xval_score = cross_val_score(lasso,X,y,cv=crossvalidation)

# compute the basic statistics
accuracy_mean = np.average(xval_score)
accuracy_std = np.std(xval_score)

lasso.fit(X,y)

importances = lasso.coef_

prec = np.average(cross_val_score(lasso,X,y,cv=crossvalidation, scoring='precision'))
    
rec = np.average(cross_val_score(lasso,X,y,cv=crossvalidation, scoring='recall'))
    
f1_metric = np.average(cross_val_score(lasso,X,y,cv=crossvalidation, scoring='f1'))
    
auc_metric = np.average(cross_val_score(lasso,X,y,cv=crossvalidation, scoring='roc_auc'))


print('\n')    
print("%40s"%method_name)
print("========================================")
print("\t  Accuracy (CV) %.3f %.3f"%(np.average(xval_score),np.std(xval_score)))
print("\t      Precision %.3f"%prec)
print("\t      Recall    %.3f"%rec)
print("\t      F1        %.3f"%f1_metric)
print("\t      AUC        %.3f"%auc_metric)
print("\n")

[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]

                                   Lasso
	  Accuracy (CV) 0.560 0.046
	      Precision 0.613
	      Recall    0.206
	      F1        0.301
	      AUC        0.575




In [30]:
importances = lasso.coef_.flatten()
importances.shape[0]

17317

In [31]:
indices = np.argsort(np.absolute(importances))[::-1]#[0:100]
indices = indices[np.absolute(importances[indices])>0.000001]

In [32]:
selected_features = importances[indices]

In [33]:
len(selected_features)

12

In [37]:
df = pd.DataFrame(data={'gene':X.columns[indices], 'importance':importances[indices]})
df.to_csv('../2022_results/importances_')

,gene,importance
0,MICAL1_rare_AD,0.521733
1,SGSM2_rare_AD,-0.216995
2,LRRC14B_rare_AD,-0.204127
3,NOD2_rare_AD,-0.185862
4,TICRR_rare_AD,0.103960
5,PSD_rare_AD,-0.089527
6,ASXL1_rare_AD,0.073988
7,RBM20_rare_AD,0.063915
8,TNKS1BP1_rare_AD,-0.036655
9,PLEKHA4_rare_AD,-0.023732


In [ ]:
n_features = len(selected_features)

fig, ax = plt.subplots(figsize=(15,7))

plt.title("Lasso Feature importances male rare AD")

ax.bar(range(n_features), importances[indices],
        color="#457b9d", yerr=None, align="center")

plt.xticks(range(n_features), X.columns[indices],rotation='-90')
#ax.get_xticklabels()[1].set_fontweight("bold")
#ax.get_xticklabels()[4].set_fontweight("bold")
#ax.get_xticklabels()[16].set_fontweight("bold")

plt.xlim([-1, len(selected_features)])#X.shape[1]])
plt.tight_layout()
plt.savefig('./Plots/Selected_AD_rare_male.png', dpi=150)
plt.show()

In [ ]:
yp_cv = cross_val_predict(lasso, X, y, cv=crossvalidation)
tn, fp, fn, tp = confusion_matrix(y, yp_cv).ravel()

#tn, fp, fn, tp = confusion_matrix(y_test, yp).ravel()

conf = [[tn,fn],[fp,tp]]

confusion = np.array(conf)

In [ ]:
ax = sns.heatmap(confusion, annot=True, cmap='Blues',fmt='g')
ax.set_ylabel('Predicted')
ax.set_xlabel('Reference')
ax.set_title('Lasso Confusion Matrix')
#ax.set_xticklabels(['Absent','Present'])
#ax.set_yticklabels(['Absent','Present'])
#plt.savefig('./Plots/Lasso Confusion matrix adj by age.png', dpi=150)